# Model

### Modelo de Cadeia de Markov Oculta: 

$m = (E, H, a, b, \pi)$ 
<br><br>
$ M = $ Tamanho do Conjunto de estados de Emissão <br>
$ N = $ Tamanho do Conjunto de estados Ocultos <br>
$ a = $ Matriz de Transição dos Estados Ocultos <br>
$ b = $ Matriz de Emissão <br>
$ \pi = $ Distribuição Inicial dos Estados Ocultos <br>



In [ ]:
m = {
    ## Variáveis
    'total_emission_sates': 3,
    'total_hidden_sates': 2,
    
    ## Probabilidades
    'transition_prob': [ #nxn
        [.75, .25],
        [.35, .65]
    ],
    'emission_prob': [ #nxm
        [.7, .2, .1],
        [.3, .1, .6]
    ],
    'inital_prab': [.8, .2] #1xn
}

# Gerador de Trajetória

### Função Geradora de Trajetória:

$O(m) = [O_1, O_2, O_3, ..., O_{T-1}, O_T]$

``` 
Initialize t <- 0, s[0] ~ pi(.)
for  t <- 0,..., T-1  do
    Draw O[t] ~ b(.|s[t])
    Draw s[t+1] ~ a(.|s[t])
```

In [2]:
from sim import generate_trajectory

O, S = generate_trajectory(100, **m)
O

array([0, 0, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 0, 1, 2, 1, 1, 0, 2, 1, 1,
       0, 1, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0,
       0, 1, 2, 1, 0, 0, 0, 2, 0, 1, 0, 0, 2, 1, 2, 0, 1, 0, 1, 0, 0, 0,
       0, 2, 2, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0,
       1, 0, 1, 0, 2, 0, 1, 2, 0, 0, 0, 0])

# Forward-Backward

#### Forward Variable: 
$\alpha_0(i) = \pi_i b_i(O_0), \;\;\;\;\; 1\leq i \leq N $ <br>
$\alpha_{t+1}(j) = b_j(O_{t+1}) \sum_{i=1}^N \alpha_t(i)a_{ij}, \;\;\;\;\; 0\leq t \leq T-2 \;\;\; 1\leq j \leq N $ <br>

```
Function Alpha(t,i)
  if t, i not in _alpha then 
    if t == 0 then
      _alpha[t, i] <- pi[i] * b[i][O[t]]
    else
      sum <- 0
      for j in N do
        sum <- sum + (Alpha(t-1, j) * a[j][i])
      _alpha[t, i] <- sum * b[i][O[t]]
  
  return _alpha[t, i]
```
#### Backward Variable:
$\beta_{T-1}(i) = 1, \;\;\;\;\; 1\leq i \leq N $ <br>
$\beta_t(i) = \sum_{j=1}^N a_{ij}b_j(O_{t+1})\beta_{t+1}(j), \;\;\;\;\; 0\leq t \leq T-2 \;\;\; 1\leq j \leq N $ <br>

```
Function Beta(t,i)
  if t, i not in _beta then 
    if t == T-1 then
      _beta[t, i] <- 1
    else
      sum <- 0
      for j in N do
        sum <- sum + (a[i][j] * b[j][O[t+1]] * Beta(t+1, j))
      _beta[t, i] <- sum
  
  return _beta[t, i]
```


In [3]:
from fb import ForwardBackward

fb = ForwardBackward(O, **m)

print(fb.alpha(2, 0))
print(fb.beta(2, 0))

print('p: ', fb.p())

0.17522399999999994
1.092448699841547e-43
p:  2.1641190278868652e-44


# Viterbi

$\delta_0(i) = \pi_i b_i(O_0), \;\;\;\;\; 1\leq i \leq N $ <br>
$\delta_t(j) = b_j(O_t) max_{1 \leq i \leq N}[a_{ij} \delta_{t-1}(i)], \;\;\;\;\; 1\leq t \leq T-1 \;\;\; 1\leq j \leq N $ <br>

```
Function Delta(t,i)
  if t, i not in _delta then 
    if t == 0 then
      _delta[t, i] <- pi[i] * b[i][O[t]]
    else
      max <- -Infinity
      for j in N do
        value <- Delta(t-1, j) * a[j][i]
        if value > max then
          max <- value
      _delta[t, i] <- max * b[i][O[t]]
  
  return _delta[t, i]
```

$\psi_0(i) = 0, \;\;\;\;\; 1\leq i \leq N $ <br>
$\psi_t(j) = argmax_{1 \leq i \leq N}[a_{ij} \delta_{t-1}(i)], \;\;\;\;\; 1\leq t \leq T-1 \;\;\; 1\leq j \leq N $ <br>

```
Function Psi(t,i)
  if t, i not in _psi then 
    if t == 0 then
      _psi[t, i] <- 0
    else
      max <- -Infinity
      argmax <- 0
      for j in N do
        value <- Psi(t-1, j) * a[j][i]
        if value > max then
          argmax <- j
      _psi[t, i] <- argmax
  
  return _psi[t, i]
```

$P^* = max_{1 \leq i \leq N}[\delta_T(i)]$ <br>
$q_{T-1}^* = argmax_{1 \leq i \leq N}[\delta_T(i)]$ <br>
$q_{t}^* = \psi_{t+1}(q_{t+1}^*), \;\;\;\;\; 0 \leq t \leq T-2$ <br>

```
Function P()
  max <- -Infinity
  for i in N do
    value <- Delta(T-1, i)
    if value > max then
      max <- value
  return max

Function q(t)
  if t = T-1 then
    argmax <- 0
    max <- -Infinity
    for i in N do
      value <- Delta(T-1, i)
      if value > max then
        argmax <- i
    return argmax
  else
    return Psi(t+1, q(t+1))
```



In [4]:
from viterbi import Viterbi

v = Viterbi(O, **m)

print(v.p_star())
print(v.q_star())

3.8287333471450946e-54
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0.]


In [5]:
print('taxa acerto:', sum(v.q_star() == S)/len(S))

taxa acerto: 0.6


# Expectation Maximization (EM)


$\xi_t(i, j) = \frac{\alpha_t(i) a_{ij}b_j(O_{t+1})\beta_{t+1}(j)}{\sum_{i=1}^N\sum_{j=1}^N\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_{t+1}(j)} $ <br>

```
Function Xi(t,i,j)
  if t, i, j not in _xi then 
    sum <- 0
    for x in N do
      for y in N do
        sum <- sum + (Alpha(t, x) * a[x][y] * b[y][O[t+1]] * Beta(t+1, y))
    _xi[t, i, j] <- (Alpha(t, i) * a[i][j] * b[j][O[t+1]] * Beta(t+1, j)) / sum

  return _xi_[t, i, j]
```

$\gamma_t(i) = \sum_{j=1}^N\xi_t(i, j) $ <br>

```
Function Gamma(t,i)
  if t, i not in _gamma then 
    sum <- 0
    for j in N do
      sum <- sum + Xi(t, i, j)
    _gamma[t, i] <- sum

  return _gamma[t, i]
```



$\bar{\pi_i} = \gamma_1(i) $ <br>

```
Function estimate_Pi(i)
  return Gamma(0, i)
```

$\bar{a_{ij}} = \frac{\sum_{t=0}^{T-2} \xi_t(i,j)}{\sum_{t=1}^{T-2} \gamma_t(i)} $ <br>

```
Function estimate_A(i,j)
  sum_num <- 0
  sum_denom <- 0

  for t <- 0,...,T do
    sum_num <- sum_num + Xi(t, i, j)
  for t <- 0,...,T do
    sum_denom <- sum_denom + Gamma(t, i)

  return sum_num / sum_denom
```
$\bar{b_j(k)} = \frac{\sum_{t=0 st O_t = v_K}^{T-2} \gamma_t(j)}{\sum_{t=1}^{T-2} \gamma_t(i)} $ <br>

```
Function estimate_B(j, k)
  sum_num <- 0
  sum_denom <- 0

  for t <- 0,...,T do
    if O[t] = k then
      sum_num <- sum_num + Gamma(t, j)
  for t <- 0,...,T do
    sum_denom <- sum_denom + Gamma(t, j)
    
  return sum_num / sum_denom
```

In [6]:
from em import ExpectationMaximization

em = ExpectationMaximization(O, **m)

print(em.estimate_pi(0))
print(em.estimate_A(0,0))
print(em.estimate_B(0,1))

0.9349509120411187
0.7829577019287107
0.24581496276688164


# Inferencia

```
Function fit(O)
    m ~ Random
    
    for n <- 0...100 do
        for i in N do 
            m[pi[i]] <- estimate_Pi(i)

            for j in N do
                m[a[i,j]] <- estimate_A(i,j)

        for k in M do 
            for j in N do
                m[b[j,k]] <- estimate_B(j,k)

    return m
```

In [7]:
from hmm import HiddenMarkovModel

hmm = HiddenMarkovModel(3, 2)
print(hmm.fit(O))

print("Real", m)
print("Esti", hmm)

(-100.8940871450565, 1)
Real {'total_emission_sates': 3, 'total_hidden_sates': 2, 'transition_prob': [[0.75, 0.25], [0.35, 0.65]], 'emission_prob': [[0.7, 0.2, 0.1], [0.3, 0.1, 0.6]], 'inital_prab': [0.8, 0.2]}
Esti {'total_emission_sates': 3, 'total_hidden_sates': 2, 'transition_prob': [[0.72, 0.28], [0.78, 0.22]], 'emission_prob': [[0.66, 0.29, 0.06], [0.25, 0.0, 0.75]], 'inital_prab': [0.71, 0.29]}
